In [ ]:
import datetime
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot
from sklearn.linear_model import LinearRegression

from statsmodels.tsa.stattools import acf, pacf
import json, os
import matplotlib.dates as mdates

from statsmodels.stats.stattools import durbin_watson
from scipy.stats import ttest_ind, normaltest
from decimal import Decimal
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
df_bw = pd.read_csv('2012_2020_18-fixed-3.csv', index_col=0).sort_index()

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
query = """
    select p.id as id, s.name as site_name, p.name as pen_name from pens p
    left join sites s
    on p.site_id = s.id
    where p.is_active is true
    and p.is_accessible_to_customer is true
    order by p.id;
"""

df_pens = rds_access_utils.extract_from_database(query)

pen_ids = [ 37, 56, 60, 66 ]
pen_infos = []

print(df_pens)

for index, pen in df_pens.iterrows():
    if pen.id in pen_ids:
            pen_infos.append((pen.id, pen.site_name, pen.pen_name))
            
pen_infos

In [ ]:
query = """
    select * from sites
"""

df_sites = rds_access_utils.extract_from_database(query)

In [ ]:
#site_name = 'Seglberget'
site_name = 'Tittelsnes'
#site_name = 'Aplavika'

site = df_sites[df_sites.name == site_name]

siteId = site['id'].values[0]

localityNo = site['government_site_number'].values[0]

manualCountsForLocality = df_bw[df_bw.localityNo == localityNo]

In [ ]:
query = """
    select * from pens 
    where site_id=%i
    and is_active is true;
""" % (siteId, )

df_pens = rds_access_utils.extract_from_database(query)

penIds = df_pens.id.values.tolist()

In [ ]:
penDFs = []

for penId in penIds:
    query = """
        select * from day_summaries
        where pen_id = %i;
    """ % (penId, )
    
    print(query)
    
    try:
        df_ab = rds_access_utils.extract_from_database(query)    
        #df_ab.index = pd.to_datetime(df_ab.captured_at)
        #df_ab = df_ab.sort_index()
        #df_daily = df_ab[columns].rolling('1D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan).fillna(method='backfill')
        #df_daily['date'] = pd.to_datetime(df_daily.index, format='%Y-%m-%d').astype(str)
        
        
        
        penDFs.append(df_ab)
    except Exception as e:
        print('Not including pen id %i' % (penId, ))
        print(e)
        continue

In [ ]:
columns = ['female_avg', 'moving_avg', 'female_moving_avg', 'moving_moving_avg', 'num_lati_fish', 'num_moving_avg_lati_fish']

siteDF = pd.concat(tuple(penDFs))
siteDF.index = pd.to_datetime(siteDF.date)
siteDF = siteDF.sort_index()
siteDF = siteDF.dropna(subset=['female_avg', 'moving_avg'])

siteAvgDF = siteDF[columns].rolling('1D').mean()

siteAvgDFStartDate = np.min(siteAvgDF.index).date()
siteAvgDFEndDate = np.max(siteAvgDF.index).date()

siteAvgDF['date'] = pd.to_datetime(siteAvgDF.index).tz_localize(None)



In [ ]:
siteAvgDF

In [ ]:
results = []

for index, manualCount in manualCountsForLocality.iterrows():   
    d = '%i-W%i' % (manualCount.year, manualCount.week)
    endDate = datetime.datetime.strptime(d + '-1', "%Y-W%W-%w").date()
    startDate = endDate - datetime.timedelta(days=6)

    if startDate >= siteAvgDFStartDate and endDate <= siteAvgDFEndDate:
        subset = (siteAvgDF.date > startDate) & (siteAvgDF.date <= endDate)
        
        numLATIFish = np.mean(siteAvgDF.num_lati_fish[subset])
        numLATIMovingAverageFish = np.mean(siteAvgDF.num_moving_avg_lati_fish[subset])
        
        aquabyteAFSubset = siteAvgDF.female_avg[subset]
        aquabyteAFSubset2 = aquabyteAFSubset[aquabyteAFSubset > 0]
        aquabyteMSubset = siteAvgDF.moving_avg[subset]
        
        aquabyteAFCount = np.max(aquabyteAFSubset)
        aquabyteAFCount2 = np.max(aquabyteAFSubset[aquabyteAFSubset > 0])
        aquabyteAFCount3 = np.max(siteAvgDF.female_moving_avg[subset])
        if aquabyteAFCount > aquabyteAFCount3 * 2:
            aquabyteAFCount = np.mean(aquabyteAFSubset)
        
        aquabyteMCount = np.max(aquabyteMSubset)
        aquabyteMCount2 = np.max(aquabyteMSubset[aquabyteMSubset > 0])
        aquabyteMCount3 = np.max(siteAvgDF.moving_moving_avg[subset])
        if aquabyteMCount > aquabyteMCount3 * 2:
            aquabyteMCount = np.mean(aquabyteMSubset)
        
        if len(aquabyteAFSubset2) < 2:
            continue
        
        results.append((startDate, manualCount.avgAdultFemaleLice, manualCount.avgMobileLice, aquabyteAFCount, aquabyteAFCount2, aquabyteAFCount3, aquabyteMCount, aquabyteMCount2, aquabyteMCount3, numLATIFish, numLATIMovingAverageFish))

results = list(zip(*results))

In [ ]:
dates = np.array(results[0])
manualCountsAF = np.array(results[1])
manualCountsM = np.array(results[2])
aquabyteCountsAF = np.array(results[3])
aquabyteCountsNonZeroAF = np.array(results[4])
aquabyteCountsMovingAF = np.array(results[5])
aquabyteCountsM = np.array(results[6])
aquabyteCountsNonZeroM = np.array(results[7])
aquabyteCountsMovingM = np.array(results[8])
numLATIFish = np.array(results[9])
numLATIMovingAverageFish = np.array(results[10])

analyzeAF = False

if analyzeAF:
    liceType = 'Adult Female'
    manualCounts = manualCountsAF
    aquabyteCounts = aquabyteCountsAF
    aquabyteCountsNonZero = aquabyteCountsNonZeroAF
    aquabyteCountsMoving = aquabyteCountsMovingAF
else:
    liceType = 'Mobile'
    manualCounts = manualCountsM
    aquabyteCounts = aquabyteCountsM
    aquabyteCountsNonZero = aquabyteCountsNonZeroM
    aquabyteCountsMoving = aquabyteCountsMovingM

X = manualCounts.reshape(-1, 1)
Y = aquabyteCountsMoving
reg = LinearRegression(fit_intercept = False).fit(X, Y)

print(reg.intercept_, reg.coef_)
print(reg.score(X, Y))

predY = reg.predict(X)

fig, ax = plt.subplots(2)

fig.set_size_inches(10, 20)

# ax[0].scatter(manualCounts, aquabyteCounts)
# ax[0].plot(manualCounts, predY, '-')
# ax[0].set_title('%s: Aquabyte vs Manual (%s)' % (site_name, liceType, ))
# ax[0].set_xlabel('Manual Count')
# ax[0].set_ylabel('Aquabyte Count')

ax[1].scatter(dates, aquabyteCounts - manualCounts)
ax[1].plot(dates, np.zeros(len(dates)), '--')
ax[1].set_title('%s: Aquabyte - Manual Delta (%s)' % (site_name, liceType, ))
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Count')

ax[0].plot(dates, manualCounts, linestyle = '--', marker = 'o', color = 'red')
ax[0].plot(dates, aquabyteCounts, linestyle = '-', marker = 'o', color = 'blue')
#ax[0].plot(dates, aquabyteCountsNonZero, linestyle = '-', marker = 'o', color = 'green')
ax[0].plot(dates, aquabyteCountsMoving, linestyle = '-', marker = 'o', color = 'purple')
ax[0].set_title('%s: Counts over time (%s)' % (site_name, liceType, ))
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Count')

ax02 = ax[0].twinx() 
#ax02.bar(dates, numLATIFish, width = 2, color = 'red', label='Num LATI Fish')
ax02.bar(dates, numLATIMovingAverageFish, width = 2, alpha = 0.25, color = 'grey', label='Num LATI Moving Avg Fish')


In [ ]:
def getQALiceCounts(pen_id):
    rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
    
    query = """
        select captured_at, annotation, annotation_metadata 
        from annotations a
        where a.pen_id = %i
        and a.is_qa = true
        and a.is_skipped = false
        and a.captured_at > '2020-03-01';
    """ % (pen_id, )

    lice_counts = rds_access_utils.extract_from_database(query)
    
    return lice_counts

In [ ]:
categories = ['ADULT_FEMALE', 'MOVING']
directions = ['LEFT', 'RIGHT', 'TOP', 'BOTTOM']
    
def getCounts(lice_counts):
    counts = {
        'NUM_FISH': {
            'LEFT': 0,
            'RIGHT': 0,
            'HAS_LICE': 0,
            'HAS_TOP_OR_BOTTOM': 0,
            'HAS_TOP_AND_BOTTOM': 0,
            'HAS_TOP_OR_BOTTOM_ADULT_FEMALE': 0,
            'HAS_TOP_AND_BOTTOM_ADULT_FEMALE': 0,
            'HAS_TOP_OR_BOTTOM_MOBILE': 0,
            'HAS_TOP_AND_BOTTOM_MOBILE': 0
        },
        'ADULT_FEMALE': {
            'TOP': 0,
            'BOTTOM': 0,
            'LEFT': 0,
            'RIGHT': 0
        },
        'MOVING': {
            'TOP': 0,
            'BOTTOM': 0,
            'LEFT': 0,
            'RIGHT': 0
        }
    }
    
    for direction in directions:
        lice_counts[direction] = 0 

        for category in categories:
            categoryDirection = category + '_' + direction

            lice_counts[categoryDirection] = 0 

    for index, lice_count in lice_counts.iterrows():   
        hasBottom = False
        hasTop = False
        hasBottomAF = False
        hasTopAF = False
        hasBottomM = False
        hasTopM = False
    
        direction = lice_count['annotation_metadata']['direction']

        lice_counts.ix[index, 'movingCountAdjusted'] = lice_count['annotation_metadata']['liceCounts']['movingCountAdjusted']
        lice_counts.ix[index, 'adultFemaleCountAdjusted'] = lice_count['annotation_metadata']['liceCounts']['adultFemaleCountAdjusted']

        leftAnnotations = lice_count['annotation']['leftCrop'] or []
        rightAnnotations = lice_count['annotation']['rightCrop'] or []
        annotations = leftAnnotations + rightAnnotations

        lice_counts.ix[index, direction] = lice_counts.ix[index, direction] + 1
        counts['NUM_FISH'][direction] = counts['NUM_FISH'][direction] + 1

        for annotation in annotations:
            category = annotation['category']
            liceLocation = annotation['liceLocation']

            categoryDirection = category + '_' + direction
            categoryDirectionLocation = category + '_' +  liceLocation

            if liceLocation == 'MIDDLE':
                if category not in counts:
                    continue
                lice_counts.ix[index, categoryDirection] = lice_counts.ix[index, categoryDirection] + 1
                counts[category][direction] = counts[category][direction] + 1
            else:
                if liceLocation == 'TOP':
                    hasTop = True
                    
                    if category == 'ADULT_FEMALE':
                        hasTopAF = True
                    elif category == 'MOVING':
                        hasTopM = True
                elif liceLocation == 'BOTTOM':
                    hasBottom = True
                    
                    if category == 'ADULT_FEMALE':
                        hasBottomAF = True
                    elif category == 'MOVING':
                        hasBottomM = True 
                
                lice_counts.ix[index, categoryDirectionLocation] = lice_counts.ix[index, categoryDirection] + 1
                counts[category][liceLocation] = counts[category][liceLocation] + 1

        if len(annotations) > 0:
            counts['NUM_FISH']['HAS_LICE'] = counts['NUM_FISH']['HAS_LICE'] + 1

        if hasTop or hasBottom:
            counts['NUM_FISH']['HAS_TOP_OR_BOTTOM'] = counts['NUM_FISH']['HAS_TOP_OR_BOTTOM'] + 1    
        if hasBottom and hasTop:
            counts['NUM_FISH']['HAS_TOP_AND_BOTTOM'] = counts['NUM_FISH']['HAS_TOP_AND_BOTTOM'] + 1
            
        if hasTopAF or hasBottomAF:
            counts['NUM_FISH']['HAS_TOP_OR_BOTTOM_ADULT_FEMALE'] = counts['NUM_FISH']['HAS_TOP_OR_BOTTOM_ADULT_FEMALE'] + 1
        if hasTopAF and hasBottomAF:
            counts['NUM_FISH']['HAS_TOP_AND_BOTTOM_ADULT_FEMALE'] = counts['NUM_FISH']['HAS_TOP_AND_BOTTOM_ADULT_FEMALE'] + 1
        
        if hasTopM or hasBottomM:
            counts['NUM_FISH']['HAS_TOP_OR_BOTTOM_MOBILE'] = counts['NUM_FISH']['HAS_TOP_OR_BOTTOM_MOBILE'] + 1
        if hasTopM and hasBottomM:
            counts['NUM_FISH']['HAS_TOP_AND_BOTTOM_MOBILE'] = counts['NUM_FISH']['HAS_TOP_AND_BOTTOM_MOBILE'] + 1   

    return counts

In [ ]:
def printStats(counts):
    af_total = 0
    m_total = 0

    for direction in directions:
        af_total = af_total + counts['ADULT_FEMALE'][direction]
        m_total = m_total + counts['MOVING'][direction]

    for direction in directions:
        if direction in [ 'TOP', 'BOTTOM' ]:
            af_ratio = counts['ADULT_FEMALE'][direction] / (counts['NUM_FISH']['LEFT'] + counts['NUM_FISH']['RIGHT'])
            m_ratio = counts['MOVING'][direction] / (counts['NUM_FISH']['LEFT'] + counts['NUM_FISH']['RIGHT'])
        else:
            af_ratio = counts['ADULT_FEMALE'][direction] / counts['NUM_FISH'][direction]
            m_ratio = counts['MOVING'][direction] / counts['NUM_FISH'][direction]
        print('%s: AF: %0.3f (%0.2f), M: %0.3f (%0.2f)' % (direction, af_ratio, counts['ADULT_FEMALE'][direction] / af_total, m_ratio, counts['MOVING'][direction] / m_total))
 
    bothPct = counts['NUM_FISH']['HAS_TOP_AND_BOTTOM'] / counts['NUM_FISH']['HAS_TOP_OR_BOTTOM']
    bothAFPct = counts['NUM_FISH']['HAS_TOP_AND_BOTTOM_ADULT_FEMALE'] / counts['NUM_FISH']['HAS_TOP_OR_BOTTOM_ADULT_FEMALE']
    bothMPct = counts['NUM_FISH']['HAS_TOP_AND_BOTTOM_MOBILE'] / counts['NUM_FISH']['HAS_TOP_OR_BOTTOM_MOBILE']
    
    print('BOTH: %0.2f, %0.2f, %0.2f' % (bothPct, bothAFPct, bothMPct))
    
    
    

In [ ]:
# Analysis of distribution of lice on fish

for pen_info in pen_infos:
    pen_id, site_name, pen_name = pen_info
    
    lice_counts = getQALiceCounts(pen_id)
    
    counts = getCounts(lice_counts)
    
    printStats(counts)
    
    print()




In [ ]:
# Analyze distribution of counts

def getAcceptedAndRejectedCounts(pen_id):
    rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))

    query = """
        select captured_at, annotation, annotator_email as qa
        from prod.crop_annotation ca
        where ca.pen_id = %i
        and ca.group_id = '%i'
        and ca.captured_at > '2020-01-01'
        and ca.service_id=1
        and ca.annotation_state_id=7;
    """ % (pen_id, pen_id)

    accepted_lice_counts = rds_access_utils.extract_from_database(query)

    accepted_lice_counts.index = accepted_lice_counts['captured_at']
    accepted_lice_counts = accepted_lice_counts.sort_index()

    query = """
        select ca1.captured_at, ca1.skip_reasons, ca2.annotation, ca1.annotator_email as qa, ca2.annotator_email as cogito
        from prod.crop_annotation ca1
        inner join prod.crop_annotation ca2
        on ca1.left_crop_url = ca2.left_crop_url
        where ca1.pen_id = %i
        and ca1.group_id = '%i'
        and ca1.captured_at > '2020-01-01'
        and ca1.service_id=1
        and ca1.annotation_state_id=6
        and ca2.pen_id = %i
        and ca2.group_id = '%i'
        and ca2.captured_at > '2020-01-01'
        and ca2.service_id=1
        and ca2.annotation_state_id=3;
    """ % (pen_id, pen_id, pen_id, pen_id)

    skipped_lice_counts = rds_access_utils.extract_from_database(query)

    return accepted_lice_counts, skipped_lice_counts

In [ ]:
def getResults(accepted_lice_counts, skipped_lice_counts):
    skippedNumFish = 0
    skippedAFCount = 0
    skippedMCount = 0
    skippedZeroCount = 0

    blurryNumFish = 0
    blurryAFCount = 0
    blurryMCount = 0
    blurryZeroCount = 0

    badOrientationNumFish = 0
    badOrientationAFCount = 0
    badOrientationMCount = 0
    badOrientationZeroCount = 0

    acceptedNumFish = 0
    acceptedAFCount = 0
    acceptedMCount = 0
    acceptedZeroCount = 0

    for index, lice_count in accepted_lice_counts.iterrows():
        annotations = lice_count['annotation']
        if not isinstance(annotations, list):
            annotations = []

        afCount = 0
        mCount = 0

        for annotation in annotations:
            if annotation['category'] == 'ADULT_FEMALE':
                afCount = afCount + 1
            elif annotation['category'] == 'MOVING':
                mCount = mCount + 1
            else:
                pass
                #print(annotation['category'])

        acceptedNumFish = acceptedNumFish + 1
        acceptedAFCount = acceptedAFCount + afCount
        acceptedMCount = acceptedMCount + mCount
        if len(annotations) == 0:
            acceptedZeroCount = acceptedZeroCount + 1

    for index, lice_count in skipped_lice_counts.iterrows():
        annotations = lice_count['annotation']
        if not isinstance(annotations, list):
            annotations = []

        afCount = 0
        mCount = 0

        for annotation in annotations:
            if annotation['category'] == 'ADULT_FEMALE':
                afCount = afCount + 1
            elif annotation['category'] == 'MOVING':
                mCount = mCount + 1
            else:
                pass
                #print(annotation['category'])

        if 'is_bad_crop_cut_off' in lice_count['skip_reasons'] or 'BAD_CROP_CUT_OFF' in lice_count['skip_reasons']:
            continue
        if 'is_obstructed' in lice_count['skip_reasons'] or 'OBSTRUCTION' in lice_count['skip_reasons']:
            continue
        if 'is_duplicate' in lice_count['skip_reasons'] or 'BAD_CROP_CUT_OFF' in lice_count['skip_reasons']:
            continue
        if 'is_bad_crop_many_fish' in lice_count['skip_reasons'] or 'BAD_CROP_MANY_FISH' in lice_count['skip_reasons']:
            continue
        if 'is_cleaner_fish' in lice_count['skip_reasons'] or 'CLEANER_FISH' in lice_count['skip_reasons']:
            continue
        if 'is_loser_fish' in lice_count['skip_reasons'] or 'LOSER_FISH' in lice_count['skip_reasons']:
            continue
        if 'is_dark' in lice_count['skip_reasons'] or 'TOO_DARK' in lice_count['skip_reasons']:
            continue

        if 'is_blurry' in lice_count['skip_reasons'] or 'BLURRY' in lice_count['skip_reasons']:
            blurryNumFish = blurryNumFish + 1
            blurryAFCount = blurryAFCount + afCount
            blurryMCount = blurryMCount + mCount
            if len(annotations) == 0:
                blurryZeroCount = blurryZeroCount + 1
        elif 'is_bad_orientation' in lice_count['skip_reasons'] or 'BAD_ORIENTATION' in lice_count['skip_reasons']:
            badOrientationNumFish = badOrientationNumFish + 1
            badOrientationAFCount = badOrientationAFCount + afCount
            badOrientationMCount = badOrientationMCount + mCount
            if len(annotations) == 0:
                badOrientationZeroCount = badOrientationZeroCount + 1
        else:
            pass
            #print(lice_count['skip_reasons'])

        skippedNumFish = skippedNumFish + 1
        skippedAFCount = skippedAFCount + afCount
        skippedMCount = skippedMCount + mCount
        if len(annotations) == 0:
            skippedZeroCount = skippedZeroCount + 1


    acceptedResults = (0,0,0,0,0,0)
    skippedResults = (0,0,0,0,0,0)
    blurryResults = (0,0,0,0,0,0)
    badOrientationResults = (0,0,0,0,0,0)
    
    if acceptedNumFish > 0:
        acceptedResults = (acceptedNumFish, acceptedAFCount / acceptedNumFish, acceptedMCount / acceptedNumFish, acceptedAFCount / acceptedNumFish / (1 - acceptedZeroCount / acceptedNumFish), acceptedMCount / acceptedNumFish  / (1 - acceptedZeroCount / acceptedNumFish), acceptedZeroCount / acceptedNumFish)
    if skippedNumFish > 0:
        skippedResults = (skippedNumFish, skippedAFCount / skippedNumFish, skippedMCount / skippedNumFish, skippedAFCount / skippedNumFish / (1 - skippedZeroCount / skippedNumFish), skippedMCount / skippedNumFish  / (1 - skippedZeroCount / skippedNumFish), skippedZeroCount / skippedNumFish)
    if blurryNumFish > 0:
        blurryResults = (blurryNumFish, blurryAFCount / blurryNumFish, blurryMCount / blurryNumFish, blurryAFCount / blurryNumFish / (1 - blurryZeroCount / blurryNumFish), blurryMCount / blurryNumFish / (1 - blurryZeroCount / blurryNumFish), blurryZeroCount / blurryNumFish)
    if badOrientationNumFish > 0:
        nonZeroCount = (badOrientationNumFish - badOrientationZeroCount)
        badOrientationResults = (badOrientationNumFish, badOrientationAFCount / badOrientationNumFish, badOrientationMCount / badOrientationNumFish, badOrientationAFCount / nonZeroCount if nonZeroCount > 0 else 0, badOrientationMCount / nonZeroCount if nonZeroCount > 0 else 0, badOrientationZeroCount / badOrientationNumFish)
    
    acceptedMinusSkippedResults = tuple(np.subtract(acceptedResults, skippedResults))
    
#     print('Accepted:        %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % acceptedResults)
#     print('Skipped:         %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % skippedResults)
#     print('Blurry:          %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % blurryResults)
#     print('Bad Orientation: %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % badOrientationResults)
#     print()
    print('AccMinusSkips:   %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % acceptedMinusSkippedResults)
    #print('BluMinusSkips:   %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % acceptedMinusSkippedResults)
    #print('OriMinusSkips:   %i fish, AF: %0.2f, M: %0.2f, Non0AF: %0.2f, Non0M: %0.2f, NonO: %0.2f' % acceptedMinusSkippedResults)
    

In [ ]:
qas = [ 'eirik@aquabyte.ai', 'embla@aquabyte.ai', 'gunnar@aquabyte.ai', 'orjan@aquabyte.ai', 'victoria@aquabyte.ai' ]

for pen_info in pen_infos:
    pen_id, site_name, pen_name = pen_info
    
    accepted_lice_counts, skipped_lice_counts = getAcceptedAndRejectedCounts(pen_id)

    print(site_name, pen_name)

    for qa in qas:
        print(qa)
        
        qa_accepted = accepted_lice_counts[accepted_lice_counts.qa == qa]
        qa_skipped = skipped_lice_counts[skipped_lice_counts.qa == qa]
        
        getResults(qa_accepted, qa_skipped)
        
        print()
    
    print()


